# PRACTICA BIG DATA ARCHITECTURE
### Pedro García

## SPRINT 1
**Utilizar una herramienta de diagramado como Google Draw o DIA para disenar y especificar el flujo de datos y herramientas utilizadas**

[DIAGRAMA](https://docs.google.com/drawings/d/1VcmcGEC22HodL4IdbReGsRJNMTiSDGk67jPnswOudNQ/edit?usp=sharing)



## SPRINT 2
**Crear un scraper en Google Collaboratory a partir de un API o de un crawler con scrapy, que descargue los datos a un archivo de formato estructurado**

He creado un scraper contra una API que informa sobre posiciones de vuelos cerca de un área. En este caso, cerca de Madrid. El resumen de su funcionalidad es el siguiente:

* Se realiza la petición
* Devuelve un JSON con la información
* Lo convierto a objeto (diccionario) de Python
* Creo el csv con los campos que he seleccionado y comprobando que vienen informados porque he visto que no siempre vienen todos los campos informados (ni siquiera aparecen)

Este csv lo usaré para la parte de HIVE

In [1]:
import requests
import json
import csv
import uuid

In [2]:
#cojo hasta un máximo de 100 vuelos que estén informados por Navegación Aérea en la zona de Madrid
#El API es este https://www.adsbexchange.com/data/
response = requests.get("https://public-api.adsbexchange.com/VirtualRadar/AircraftList.json?lat=40.4167&lng=-3.70325&fDstL=0&fDstU=100")
print (response.status_code)
print (response.content)

200
b'{"src":1,"feeds":[{"id":1,"name":"From Cons","polarPlot":false}],"srcFeed":1,"showSil":true,"showFlg":true,"showPic":true,"flgH":20,"flgW":85,"acList":[{"Id":4221706,"Rcvr":1,"HasSig":true,"Sig":156,"Icao":"406B0A","Bad":false,"Reg":"G-STBK","FSeen":"\\/Date(1546362490015)\\/","TSecs":115,"CMsgs":47,"Alt":8100,"GAlt":8593,"InHg":30.4133873,"AltT":0,"Call":"BAW461","Lat":40.666638,"Long":-3.455762,"PosTime":1546362603777,"Mlat":false,"Tisb":false,"Spd":274.9,"Trak":39.5,"TrkH":false,"Type":"B773","Mdl":"Boeing 777 336ER","Man":"Boeing","CNum":"42121","From":"LEMD Madrid Barajas, Spain","To":"EGLL London Heathrow, United Kingdom","Op":"British Airways","OpIcao":"BAW","Sqk":"5326","Help":false,"Vsi":3328,"VsiT":0,"Dst":34.78,"Brng":36.9,"WTC":3,"Species":1,"Engines":"2","EngType":3,"EngMount":0,"Mil":false,"Cou":"United Kingdom","HasPic":false,"Interested":false,"FlightsCount":0,"Gnd":false,"SpdTyp":0,"CallSus":false,"Trt":2,"Year":"2014"},{"Id":5022552,"Rcvr":1,"HasSig":true,"Sig":

In [3]:
#lo convierto a objeto python
flights = response.json()

In [4]:
#compruebo lo que he capturado
print (flights)

{'src': 1, 'feeds': [{'id': 1, 'name': 'From Cons', 'polarPlot': False}], 'srcFeed': 1, 'showSil': True, 'showFlg': True, 'showPic': True, 'flgH': 20, 'flgW': 85, 'acList': [{'Id': 4221706, 'Rcvr': 1, 'HasSig': True, 'Sig': 156, 'Icao': '406B0A', 'Bad': False, 'Reg': 'G-STBK', 'FSeen': '/Date(1546362490015)/', 'TSecs': 115, 'CMsgs': 47, 'Alt': 8100, 'GAlt': 8593, 'InHg': 30.4133873, 'AltT': 0, 'Call': 'BAW461', 'Lat': 40.666638, 'Long': -3.455762, 'PosTime': 1546362603777, 'Mlat': False, 'Tisb': False, 'Spd': 274.9, 'Trak': 39.5, 'TrkH': False, 'Type': 'B773', 'Mdl': 'Boeing 777 336ER', 'Man': 'Boeing', 'CNum': '42121', 'From': 'LEMD Madrid Barajas, Spain', 'To': 'EGLL London Heathrow, United Kingdom', 'Op': 'British Airways', 'OpIcao': 'BAW', 'Sqk': '5326', 'Help': False, 'Vsi': 3328, 'VsiT': 0, 'Dst': 34.78, 'Brng': 36.9, 'WTC': 3, 'Species': 1, 'Engines': '2', 'EngType': 3, 'EngMount': 0, 'Mil': False, 'Cou': 'United Kingdom', 'HasPic': False, 'Interested': False, 'FlightsCount': 0,

In [5]:
#miro las claves del JSON convertidas a Diccionario para ver dónde buscar
print(flights.keys())

dict_keys(['src', 'feeds', 'srcFeed', 'showSil', 'showFlg', 'showPic', 'flgH', 'flgW', 'acList', 'totalAc', 'lastDv', 'shtTrlSec', 'stm'])


In [6]:
#miro los campos informados en el primer elemento
#Los campos vienen explicados en https://www.adsbexchange.com/datafields/
flights['acList'][0]

{'Id': 4221706,
 'Rcvr': 1,
 'HasSig': True,
 'Sig': 156,
 'Icao': '406B0A',
 'Bad': False,
 'Reg': 'G-STBK',
 'FSeen': '/Date(1546362490015)/',
 'TSecs': 115,
 'CMsgs': 47,
 'Alt': 8100,
 'GAlt': 8593,
 'InHg': 30.4133873,
 'AltT': 0,
 'Call': 'BAW461',
 'Lat': 40.666638,
 'Long': -3.455762,
 'PosTime': 1546362603777,
 'Mlat': False,
 'Tisb': False,
 'Spd': 274.9,
 'Trak': 39.5,
 'TrkH': False,
 'Type': 'B773',
 'Mdl': 'Boeing 777 336ER',
 'Man': 'Boeing',
 'CNum': '42121',
 'From': 'LEMD Madrid Barajas, Spain',
 'To': 'EGLL London Heathrow, United Kingdom',
 'Op': 'British Airways',
 'OpIcao': 'BAW',
 'Sqk': '5326',
 'Help': False,
 'Vsi': 3328,
 'VsiT': 0,
 'Dst': 34.78,
 'Brng': 36.9,
 'WTC': 3,
 'Species': 1,
 'Engines': '2',
 'EngType': 3,
 'EngMount': 0,
 'Mil': False,
 'Cou': 'United Kingdom',
 'HasPic': False,
 'Interested': False,
 'FlightsCount': 0,
 'Gnd': False,
 'SpdTyp': 0,
 'CallSus': False,
 'Trt': 2,
 'Year': '2014'}

In [7]:
#creo el fichero csv
flightFile = open('flightsMAD.csv', 'w')
with flightFile:
    fileWriter = csv.writer(flightFile)
    
    #encabezado, realmente para el LOAD DATA de HIVE no usamos el encabezado
    #pero lo dejo a modo informativo en la creación del csv por si sirve para otra cosa, para el HIVE no necesito la cabecera
    #fileWriter.writerow(["Callsign", "OperatedBy", "From", "To", "Aircraft", "Altitude", "Latitude", "Longitude", "PositionTime","Speed","VerticalSpeed"])
    
    #array para los registros de las filas
    flightData = []
    for flight in flights['acList']:

        
        #la clave es un UUID usando como semilla el callsign + position time
        flightData.append(uuid.uuid4())

        #no siempre vienen informados los campos, así que siempre los compruebo
        #hay algunos que no compruebo porque siempre vienen informados
        #y al menos introduzco algo de información para saber si tuve o no datos


        if 'Call' in flight.keys():
            flightData.append(flight['Call'])
        else:
            flightData.append('NO_CALL')
        if 'Op' in flight.keys():
            flightData.append(flight["Op"])
        else:    
            flightData.append('NO_OP')
        if 'From' in flight.keys():
            #elimino las comas que hay dentro del campo ya que se confunden con el separador de CSV
            flightData.append((flight["From"]).replace(',', ''))
        else:    
            flightData.append('NO_FROM')
        if 'To' in flight.keys():
            flightData.append((flight["To"]).replace(',', ''))
        else:    
            flightData.append('NO_TO')
        if 'Mdl' in flight.keys():
            flightData.append((flight["Mdl"]).replace(',', ''))
        else:
            flightData.append('NO_AIRCRAFT')
        if 'Alt' in flight.keys():    
            flightData.append(flight["Alt"])
        else:
            flightData.append('NO_ALTITUDE')
        if 'Lat' in flight.keys():        
            flightData.append(flight["Lat"])
        else:
            flightData.append('NO_LATITUDE')
        if 'Long' in flight.keys():            
            flightData.append(flight["Long"])
        else:    
            flightData.append('NO_LONGITUDE')
                               
        flightData.append(flight["PosTime"])
        
        if 'Spd' in flight.keys():                                    
            flightData.append(flight["Spd"])
        else:
            flightData.append('NO_SPEED')
        if 'Vsi' in flight.keys():       
            flightData.append(flight["Vsi"])
        else:    
            flightData.append('NO_VSI')

        #escribo la fila en el csv    
        fileWriter.writerow(flightData)
        
        #limpio el array para acumular los campos del siguiente vuelo
        flightData.clear()
        

In [8]:
#me bajo el fichero
flightFile.close()
from google.colab import files
files.download('flightsMAD.csv')

ModuleNotFoundError: No module named 'google'

## BONUS
**Utilizar HIVE para categorizar los datos y hacer un par de queries con el command line para extraer datos a un fichero.**




### 1.De CSV a HIVE

* En un primer paso he creado la tabla en HIVE, (debería haber creado antes un DATABASE) 


```
CREATE TABLE IF NOT EXISTS flights (id_flight STRING, 
 callsign VARCHAR(20), 
 operated_by VARCHAR(100), 
 origin VARCHAR(200), 
 destination VARCHAR(200), 
 aircraft VARCHAR(100), 
 altitude INT, 
 latitude DECIMAL(9,6), 
 longitude DECIMAL(9,6), 
 positionTime BIGINT, 
 speed DECIMAL(4,1), 
 vsi INT)
 ROW FORMAT DELIMITED FIELDS TERMINATED BY ',';
```

* He creado un CSV con lo capturado por el scrapper del  **SPRINT 2**

* Lo he cargado en HIVE con:

```
LOAD DATA LOCAL INPATH '/opt/flightsMAD.csv' INTO TABLE flights;
```

* Esta es la captura de pantalla tras el **_select * from flights;_**


[Captura imagen de carga del CSV a Hive](https://drive.google.com/open?id=1i3rJvAv5GMCjAn4H9ZXdL4aQKhVR5p2x)


### 2.Consulta a HIVE exportándolo como fichero al HDFS de Hadoop

* En este segundo paso, he realizado una consulta a la tabla **_flights_** y lo he incluido como un fichero de texto en HDFS en una carpeta previamente creada en HDFS.

* Vamos a seleccionar los vuelos cercanos a MAD cuya velocidad sea mayor a 400 nudos.

[Captura imagen del Select](https://drive.google.com/open?id=1KsWidbOhFk-tucRwJLgo9QCb1Vdk0YfY)

* Ejecutamos:


```
./hive -S -e 'select callsign, origin, destination, altitude, latitude, longitude, speed from flights where speed > 400' | $HADOOP_PREFIX/bin/hdfs dfs -put flights400MAD.txt vuelos/flights400MAD.txt

```

[Captura imagen inserción fichero en HDFS desde consulta en HIVE](https://drive.google.com/open?id=1yvBIOJA_w0GMLZXZKzVYHfCi2q-dcBS7)

* Visualizar el fichero desde HDFS

[Consulta del fichero en HDFS](https://drive.google.com/open?id=1hbMYg993JbIapv-0yLWF_j9wEUayzmrJ)


### 3. Desde un cliente Python a HIVE

Usando el mismo ejemplo del Scrapper de vuelos, voy a utilizar un cliente Python para insertar directamente os vuelos en HIVE

En el siguiente enlace están los registros de la tabla **_flights_** antes de actualizarla con nuevos vuelos

[Vuelos insertados previamente](https://drive.google.com/open?id=1fWDMVPhnDuTN01pg4sbq3Nj7Zxpu4RpZ)

A continuación se muestra el programa cliente de Python que inserta los nuevos vuelos recogidos desde el API

In [0]:
import requests
import json
import csv
import uuid
from pyhive import hive

In [0]:
# Conexión a HIVE
cursor = hive.connect('localhost').cursor()

In [0]:
# comprobación de que todo va bien y puedo recuperar registros ya insertados desde el CSV el punto 1 de BONUS
cursor.execute('select * from flights')
print (cursor.fetchall())

** Captura del resultado que devuelve  la consulta a HIVE desde Python**

[Vuelos ya presentes en HIVE](https://drive.google.com/open?id=1fWDMVPhnDuTN01pg4sbq3Nj7Zxpu4RpZ)




In [0]:
# consulta al API de vuelos 
response = requests.get("https://public-api.adsbexchange.com/VirtualRadar/AircraftList.json?lat=40.4167&lng=-3.70325&fDstL=0&fDstU=100")
print (response.status_code)
print (response.content)

In [0]:
#lo convierto a objeto python
flights = response.json()

In [0]:

# para cada vuelo recibido, compongo el INSERT y lo ejecuto para grabarlo en HIVE

for flight in flights['acList']:
    
    insert = 'INSERT INTO flights VALUES('
    
    insert = insert + '\'' + str(uuid.uuid4()) +  '\'' + ','


    if 'Call' in flight.keys():
        insert = insert + '\'' + str(flight['Call']) + '\'' + ', '
    else:
        insert = insert + '\'' + 'NO_CALL' + '\'' + ','
    if 'Op' in flight.keys():
        insert = insert + '\'' + str(flight["Op"]) + '\'' + ', '
    else:    
        insert = insert + '\'' + 'NO_OP' + '\'' + ', '
    if 'From' in flight.keys():
    #elimino las comas que hay dentro del campo ya que se confunden con el separador del insert
        insert = insert + '\'' + flight["From"].replace(',', '') + '\'' + ', ' 
    else:    
        insert = insert + '\'' + 'NO_FROM' + '\'' + ', '
    if 'To' in flight.keys():
        insert = insert + '\'' + flight["To"].replace(',', '') + '\'' + ', '
    else:    
        insert = insert + '\'' + 'NO_TO' + '\'' + ', '
    if 'Mdl' in flight.keys():
        insert = insert + '\'' + flight["Mdl"].replace(',', '') + '\'' + ', '
    else:
        insert = insert + '\'' + 'NO_AIRCRAFT' + '\'' + ', '
    if 'Alt' in flight.keys():    
        insert = insert + str(flight["Alt"]) + ', '
    else:
        insert = insert + 'NO_ALTITUDE' + ', '
    if 'Lat' in flight.keys():        
        insert = insert + str(flight["Lat"]) + ', '
    else:
        insert = insert + 'NO_LATITUDE' + ', '
    if 'Long' in flight.keys():            
        insert = insert + str(flight["Long"]) + ', '
    else:    
        insert = insert + 'NO_LONGITUDE' + ', '
                               
    insert = insert + str(flight["PosTime"]) + ', '
        
    if 'Spd' in flight.keys():                                    
        insert = insert + str(flight["Spd"]) + ', '
    else:
        insert = insert + 'NO_SPEED' + ', '
    if 'Vsi' in flight.keys():       
        insert = insert + str(flight["Vsi"])
    else:    
        insert = insert + 'NO_VSI'
    
    #fin de la composición del INSERT
    insert = insert + ')'
    
    print (insert)
    
    #ejecuto la consulta para insertar el registro
    cursor.execute(insert)

### Ejemplo de alguno de los INSERT generados desde el cliente Python e insertados en HIVE



```
INSERT INTO flights VALUES('23ee8506-1a70-4fc2-8d01-0259ff716d72','KAL914', 'Korean Air', 'LEMD Madrid Barajas Spain', 'RKSI Incheon Seoul South Korea', 'Boeing 777 3B5ER', 7225, 40.678556, -3.443215, 1531248433115, 283.0, 2624)

INSERT INTO flights VALUES('c30078f8-4800-40dc-babb-d3386657145c','KLM1706', 'KLM Royal Dutch Airlines', 'LEMD Madrid Barajas Spain', 'EHAM Amsterdam Airport Schiphol Netherlands', 'Boeing 737NG 9K2/W', 12675, 40.778829, -3.331042, 1531248433115, 344.8, 1088)

INSERT INTO flights VALUES('8b5791a2-1f42-4c29-9a1c-a5c46cd4c19b','RYR27ZU', 'Ryanair', 'NO_FROM', 'NO_TO', 'Boeing 737NG 8AS/W', 15675, 40.634079, -3.267883, 1531248433115, 371.4, 1024)

INSERT INTO flights VALUES('2f8c9055-0a22-4979-a658-0351ddad4830','AEA7306', 'Air Europa', 'LEMD Madrid Barajas Spain', 'LEVX Vigo Spain', 'Embraer EMB-195 LR', 22350, 41.07532, -4.37181, 1531248433115, 393.3, 1152)

```

### Consulta en HIVE para mostrar todos los registros insertados

[Listado de los vuelos insertados tanto mediante el CSV como desde el cliente Python](https://drive.google.com/open?id=1KQ6myyTrZcE79tMPOOwvpU83cMpmXZET)



